### HJB

In [1]:
using Pkg
# Pkg.add("OptimalControl")
# Pkg.add("NLPModelsIpopt")
# Pkg.add("Plots")

In [6]:
using DifferentialEquations
using Plots

# Parámetros
S₀ = 0.9
I₀ = 0.1
R₀ = 0
V₀ = 0
Z₀ = 0.1
β = 0.8
ϕ = 0.5
γ = 0.3
λ_p = 1.0
T = 10.0
n_steps = 30
dt = T / n_steps
λ_v = 90
u_max = 1.0  # Valor máximo de u

# Rango espacial para las variables de estado
S_range = LinRange(0, 1, 20)  # Valores para S
V_range = LinRange(0, 1, 20)  # Valores para V
I_range = LinRange(0, 1, 20)  # Valores para I
R_range = LinRange(0, 1, 20)  # Valores para R
Z_range = LinRange(0, 1, 20)  # Valores para Z

# Inicializar la función valor V(x, t) como una matriz de ceros
V_bar = zeros(length(S_range), length(V_range), length(I_range), length(R_range), length(Z_range), n_steps)

# Condición terminal: V(x, T) = λ_p Z(T)
for i in 1:length(S_range), j in 1:length(V_range), k in 1:length(I_range), l in 1:length(R_range), m in 1:length(Z_range)
    V_bar[i, j, k, l, m, end] = λ_p * Z_range[m]
end

# Resolver hacia atrás en el tiempo
for t in (n_steps-1):-1:1
    for i in 1:length(S_range), j in 1:length(V_range), k in 1:length(I_range), l in 1:length(R_range), m in 1:length(Z_range)
        S, V, I, R, Z = S_range[i], V_range[j], I_range[k], R_range[l], Z_range[m]

        # Derivadas parciales aproximadas (diferencias finitas)
        V_s = (i > 1 ? (V_bar[i, j, k, l, m, t+1] - V_bar[i-1, j, k, l, m, t+1]) / (S_range[2] - S_range[1]) : 0.0)
        V_v = (j > 1 ? (V_bar[i, j, k, l, m, t+1] - V_bar[i, j-1, k, l, m, t+1]) / (V_range[2] - V_range[1]) : 0.0)
        V_i = (k > 1 ? (V_bar[i, j, k, l, m, t+1] - V_bar[i, j, k-1, l, m, t+1]) / (I_range[2] - I_range[1]) : 0.0)
        V_r = (l > 1 ? (V_bar[i, j, k, l, m, t+1] - V_bar[i, j, k, l-1, m, t+1]) / (R_range[2] - R_range[1]) : 0.0)
        V_z = (m > 1 ? (V_bar[i, j, k, l, m, t+1] - V_bar[i, j, k, l, m-1, t+1]) / (Z_range[2] - Z_range[1]) : 0.0)

        # Control óptimo v*
        dV_dz = -V_z * exp(0.234 * log(exp(360 * (I - Z)) + 1))
        v_opt = (dV_dz > 0 ? 1.0 : 0.0)

        # Definición de ν_opt (basado en el problema)
        ν_opt = v_opt  # Esto puede cambiar si ν_opt tiene otra lógica

        # Control óptimo u*
        delta_u = V_v - V_s  # ∂V/∂V - ∂V/∂S
        u_opt = (λ_v > delta_u ? u_max : 0.0)

        # Función de costo minimizada
        min_value = -λ_v * u_opt * S +
                    V_s * (-β * S * I - u_opt * S) +
                    V_v * (u_opt * S - β * (1 - ϕ) * V * I) +
                    V_i * (β * S * I + β * (1 - ϕ) * V * I - γ * I) +
                    V_r * (γ * I) +
                    V_z * (log(exp(400 * (β * S * I + β * (1 - ϕ) * V * I - γ * I)) + 1) / 400) * 
                          (1 - ν_opt * exp(0.234 * log(exp(360 * (I - Z)) + 1)))

        # Actualizar función valor
        V_bar[i, j, k, l, m, t] = V_bar[i, j, k, l, m, t+1] - dt * min_value
    end
end

# Seleccionar y mostrar valores de V_bar para ciertas dimensiones
selected_V_bar = V_bar[:, 1, :, 1, 1, 1]

println("Valores de V_bar para S e I:")
for i in 1:size(selected_V_bar, 1)
    for j in 1:size(selected_V_bar, 2)
        println("S = $(S_range[i]), I = $(I_range[j]), V_bar = $(selected_V_bar[i, j])")
    end
end


Valores de V_bar para S e I:
S = 0.0, I = 0.0, V_bar = 0.0
S = 0.0, I = 0.05263157894736842, V_bar = 0.0
S = 0.0, I = 0.10526315789473684, V_bar = 0.0
S = 0.0, I = 0.15789473684210525, V_bar = 0.0
S = 0.0, I = 0.21052631578947367, V_bar = 0.0
S = 0.0, I = 0.2631578947368421, V_bar = 0.0
S = 0.0, I = 0.3157894736842105, V_bar = 0.0
S = 0.0, I = 0.3684210526315789, V_bar = 0.0
S = 0.0, I = 0.42105263157894735, V_bar = 0.0
S = 0.0, I = 0.47368421052631576, V_bar = 0.0
S = 0.0, I = 0.5263157894736842, V_bar = 0.0
S = 0.0, I = 0.5789473684210527, V_bar = 0.0
S = 0.0, I = 0.631578947368421, V_bar = 0.0
S = 0.0, I = 0.6842105263157895, V_bar = 0.0
S = 0.0, I = 0.7368421052631579, V_bar = 0.0
S = 0.0, I = 0.7894736842105263, V_bar = 0.0
S = 0.0, I = 0.8421052631578947, V_bar = 0.0
S = 0.0, I = 0.8947368421052632, V_bar = 0.0
S = 0.0, I = 0.9473684210526315, V_bar = 0.0
S = 0.0, I = 1.0, V_bar = 0.0
S = 0.05263157894736842, I = 0.0, V_bar = 19888.8122566043
S = 0.05263157894736842, I = 0.052631

In [24]:
using Plots

# Seleccionar los valores de V_bar para ciertas dimensiones
selected_V_bar = V_bar[:, 1, :, 1, 1, 1]

# Crear un mapa de calor para V_bar con S e I como ejes
heatmap(S_range, I_range, selected_V_bar',
    xlabel="S",
    ylabel="I",
    title="Mapa de calor de V_bar (S vs I)",
    color=:viridis)

savefig("svi.png")

"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\svi.png"

In [25]:
selected_V_bar = V_bar[:, :, 1, 1, 1, 1]  # Fijamos I, R y Z
heatmap(S_range, V_range, selected_V_bar',
    xlabel="S",
    ylabel="V",
    title="Mapa de calor de V_bar (S vs V)",
    color=:viridis)

savefig("svv.png") 

"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\svv.png"

In [26]:
selected_V_bar = V_bar[:, 1, 1, :, 1, 1]  # Fijamos V, I y Z
heatmap(S_range, R_range, selected_V_bar',
    xlabel="S",
    ylabel="R",
    title="Mapa de calor de V_bar (S vs R)",
    color=:plasma)

    savefig("svr.png")

"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\svr.png"

In [27]:
selected_V_bar = V_bar[:, 1, 1, 1, :, 1]  # Fijamos V, I y R
heatmap(S_range, Z_range, selected_V_bar',
    xlabel="S",
    ylabel="Z",
    title="Mapa de calor de V_bar (S vs Z)",
    color=:inferno)

    savefig("svz.png")

"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\svz.png"

In [29]:
selected_V_bar = V_bar[1, :, :, 1, 1, 1]  # Fijamos S, R y Z
heatmap(V_range, I_range, selected_V_bar',
    xlabel="V",
    ylabel="I",
    title="Mapa de calor de V_bar (V vs I)",
    color=:coolwarm)

    savefig("vvi.png")

"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\vvi.png"

In [30]:
selected_V_bar = V_bar[1, :, 1, :, 1, 1]  # Fijamos S, I y Z
heatmap(V_range, R_range, selected_V_bar',
    xlabel="V",
    ylabel="R",
    title="Mapa de calor de V_bar (V vs R)",
    color=:magma)


    savefig("vvr.png")    

"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\vvr.png"

In [31]:
selected_V_bar = V_bar[1, :, 1, 1, :, 1]  # Fijamos S, I y R
heatmap(V_range, Z_range, selected_V_bar',
    xlabel="V",
    ylabel="Z",
    title="Mapa de calor de V_bar (V vs Z)",
    color=:cividis)

    savefig("vvz.png")

"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\vvz.png"

In [32]:
selected_V_bar = V_bar[1, 1, :, :, 1, 1]  # Fijamos S, V y Z
heatmap(I_range, R_range, selected_V_bar',
    xlabel="I",
    ylabel="R",
    title="Mapa de calor de V_bar (I vs R)",
    color=:viridis)


    savefig("ivr.png")


"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\ivr.png"

In [33]:
selected_V_bar = V_bar[1, 1, :, 1, :, 1]  # Fijamos S, V y R
heatmap(I_range, Z_range, selected_V_bar',
    xlabel="I",
    ylabel="Z",
    title="Mapa de calor de V_bar (I vs Z)",
    color=:plasma)


    savefig("ivz.png")

"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\ivz.png"

In [34]:
selected_V_bar = V_bar[1, 1, 1, :, :, 1]  # Fijamos S, V y I
heatmap(R_range, Z_range, selected_V_bar',
    xlabel="R",
    ylabel="Z",
    title="Mapa de calor de V_bar (R vs Z)",
    color=:inferno)

    savefig("rvz.png")    

"c:\\Users\\rodri\\Desktop\\Universidad\\primavera2024\\Control optimo\\proyecto_co\\rvz.png"